### MAST30034: Applied Data Science Project 1
---
# Data Analysis: FHVHV Dataset
#### Xavier Travers (1178369)

Performs data analysis to relate the COVID dataset and FHVHV aggregate dataset

In [1]:
# imports used throughout this notebook
from pyspark.sql import DataFrame, Column, Window
from pyspark.sql.types import IntegerType
from pyspark.sql import functions as F
import matplotlib.pyplot as plt
import pandas as pd
import os
import sys

# add homemade helpers
sys.path.insert(1, '../scripts')
import helpers.join_helpers as jh
import helpers.plot_helpers as ph

DEBUGGING = False

In [2]:
from pyspark.sql import SparkSession

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName('MAST30034 XT Project 1')
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.sql.repl.eagerEval.enabled', True) 
    .config('spark.sql.parquet.cacheMetadata', 'true')
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

22/08/14 03:03:57 WARN Utils: Your hostname, Polaris resolves to a loopback address: 127.0.1.1; using 172.26.235.73 instead (on interface eth0)
22/08/14 03:03:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/14 03:03:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/14 03:03:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/08/14 03:03:59 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/08/14 03:03:59 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/08/14 03:03:59 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/08/14 03:03:59 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
22/08/14 03:03:59 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
22/08/14 03:03:59 WARN Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 4047.


In [3]:
# read in the aggregated yellow tlc data
tlc_pu_df = spark.read.parquet('../data/curated/tlc/aggregated/yellow/by_pu')
tlc_pu_df.limit(5)

week_year,week_month,week_ending,week_index,timeline,pu_borough,num_*,tot_passengers,daily_avg_passengers,avg_passengers,tot_trip_distance,daily_avg_trip_distance,avg_trip_distance,tot_hours_elapsed,daily_avg_hours_elapsed,avg_hours_elapsed
2020,5,2020-05-16,124,post,Brooklyn,986,1334.0,190.57142857142858,1.3529411764705883,3984.920000000005,569.2742857142864,4.041501014198788,316.52833333333365,45.218333333333376,0.32102265043948647
2021,1,2021-01-09,158,post,Staten Island,55,57.0,8.142857142857142,1.0363636363636364,1663.89,237.69857142857146,30.252545454545455,45.27888888888889,6.468412698412699,0.8232525252525253
2020,8,2020-08-15,137,post,Bronx,1263,1511.0,215.85714285714286,1.196357878068092,3173.2900000000004,453.3271428571429,2.5125019794140937,435.9388888888894,62.276984126984196,0.34516143221606443
2020,4,2020-05-02,122,post,Bronx,557,708.0,101.14285714285714,1.2710951526032317,2042.720000000001,291.817142857143,3.6673608617594273,173.4888888888888,24.78412698412697,0.3114701775384
2020,6,2020-06-06,127,post,Manhattan,78025,104586.0,14940.857142857143,1.340416533162448,191468.2500000006,27352.60714285723,2.453934636334516,15841.168611110981,2263.0240873015687,0.2030268325679075


In [4]:
# read in the aggregated covid data
covid_df = spark.read.parquet('../data/curated/virals/covid/aggregated/cases-by-week')
covid_df.limit(5)

week_year,week_month,week_ending,week_index,timeline,borough,tot_cases,daily_avg_cases,tot_deaths,daily_avg_deaths,tot_hospitalised,daily_avg_hospitalised
2021,8,2021-08-21,190,neither,Staten Island,899.0,128.42857142857142,9.0,1.2857142857142858,65.0,9.285714285714286
2021,11,2021-11-06,201,neither,Staten Island,363.0,51.857142857142854,9.0,1.2857142857142858,28.0,4.0
2020,5,2020-05-16,124,post,Staten Island,241.0,34.42857142857143,37.0,5.285714285714286,52.0,7.428571428571429
2021,11,2021-11-13,202,neither,Staten Island,488.0,69.71428571428571,10.0,1.4285714285714286,27.0,3.857142857142857
2020,12,2020-12-05,153,post,Staten Island,2319.0,331.2857142857143,32.0,4.571428571428571,164.0,23.428571428571427


In [5]:
# join the datasets by week_index
# TODO: redo join_by_week_by_borough
joined_pu_df = jh.join_by_week_by_borough(tlc_pu_df, covid_df, 'tot_cases')
joined_pu_df.limit(5)

AnalysisException: USING column `do_borough` cannot be resolved on the left side of the join. The left-side columns: [pu_borough, week_year, week_month, week_ending, week_index, timeline, num_*, tot_passengers, daily_avg_passengers, avg_passengers, tot_trip_distance, daily_avg_trip_distance, avg_trip_distance, tot_hours_elapsed, daily_avg_hours_elapsed, avg_hours_elapsed, pu_tot_cases, prev_week_index]

In [ ]:
# plot a time-series histogram to show how average weekly trips changes per week
ph.group_plot(joined_pu_df.toPandas(), 'week_index', 'avg_trips',
    'do_borough', filename_prefix = 'yellow-covid', kind = 'scatter',
    xlabel = 'MMWR Weeks Since 2018', 
    ylabel = 'Average Weekly Incoming Trips', logy=True)

In [ ]:
borough_part = Window.partitionBy('do_borough')
joined_df.withColumn('min_avg_trips', F.min('avg_trips').over(borough_part))\
    .where(F.col('avg_trips') == F.col('min_avg_trips'))\
    .drop('min_avg_trips')

The above quite clearly demonstrates the prevalent COVID-19 dip at around March-April 2020 when lockdowns started and predicted outcomes (for everything, were at an all-time low).

In [ ]:
# plot a general/dirty scatter plot of avg wkly trips vs cases from prev wk
ph.group_plot(joined_df.toPandas(), 'do_tot_cases', 'avg_trips',
    'do_borough', filename_prefix = 'yellow-covid', 
    xlabel = 'Previous Week New Cases', 
    ylabel = 'Average Weekly Incoming Trips',
    logx=True, logy=True)